# Mission to Mars

## Step 1 - Scraping

In [1]:
# Dependencies
import pandas as pd
import requests
import time
from splinter import Browser
from bs4 import BeautifulSoup
import tweepy
import TweepyCredentials # twitter keys and tokens
import pymongo


### NASA Mars News 

In [2]:
# URL of NASA Mars News to be scraped for latest news and paragraph title
url_NASA_Mars_News = 'https://mars.nasa.gov/news/'

# Setting up splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)
browser.visit(url_NASA_Mars_News)

html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')

In [3]:
# Latest News Title from NASA Mars News Site
news_title = news_soup.find_all('div', class_='content_title')
print(news_title[0].text)

Mars Mission Sheds Light on Habitability of Distant Planets


In [4]:
# Latest News Paragraph Text from NASA Mars News Site
news_p = news_soup.find_all('div', class_='article_teaser_body')
print(news_p[0].text)

How long might a rocky, Mars-like planet be habitable if it were orbiting a red dwarf star?


### JPL Mars Space Images - Featured Image

In [5]:
# URL of JPL Mars Space Image to be scraped for featured image
url_JPL_images = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# Setting up splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)
browser.visit(url_JPL_images)

In [6]:
# Browse through the pages
time.sleep(5)

# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [7]:
time.sleep(5)

# Find the more info button and click that
more_info_elem = browser.find_link_by_partial_text('more info')
more_info_elem.click()

In [8]:
# Using BeautifulSoup create an object and parse with 'html.parser'
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [9]:
# find the relative image url
img_url_rel = img_soup.find('figure', class_='lede').find('img')['src']
img_url_rel

'/spaceimages/images/largesize/PIA19980_hires.jpg'

In [10]:
# Use the base url to create an absolute url
JPL_link = 'https://www.jpl.nasa.gov'
featured_image_url = JPL_link + img_url_rel
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19980_hires.jpg'

### Mars Weather

In [11]:
# Twitter API Keys
consumer_key = TweepyCredentials.consumer_key
consumer_secret = TweepyCredentials.consumer_secret
access_token = TweepyCredentials.access_token
access_token_secret = TweepyCredentials.access_token_secret

In [12]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [13]:
# Target User
target_user = "@MarsWxReport"

In [14]:
# Retrive latest tweet
tweet = api.user_timeline(target_user)
mars_weather = tweet[0]['text']
mars_weather

'Sol 1927 (Jan 07, 2018), Sunny, high -24C/-11F, low -79C/-110F, pressure at 7.74 hPa, daylight 05:46-17:30'

### Mars Facts

In [15]:
# URL of Mars facts to scrape the table containing facts about the planet
url_Mars_Facts = 'https://space-facts.com/mars/'

df = pd.read_html(url_Mars_Facts)[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [16]:
# Convert the dataframe to HTML table string
mars_profile_html = df.to_html()
mars_profile_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>

### Mars Hemisphere

In [17]:
# Scapping of  USGS Astrogeology site to obtain high resolution images for each of Mars hemispheres.
url_USGS = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# Setting up splinter
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path)
browser.visit(url_USGS)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [18]:
# Create an empty list to hold dictionaries of hemisphere title with the image url string
hemisphere_image_urls = []

# Get a list of all of the hemispheres
links = browser.find_by_css("a.product-item")

# Loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()

In [19]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [20]:
browser.quit()